## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd

data = pd.read_csv('/datasets/data.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

Всего 21525 записей. Имеются пропуски в данных и некорректные значения.

| Признак | Тип | Описание |
|:-|:-|:-|
| children |int64 | категориальное значение (дети), должно быть >= 0 |
| days_employed |float64 | количественное значение (стаж в днях), должно быть >= 0, имеются пропуски |
| dob_years |int64 | категориальное значение (возраст в годах), должно быть >= 18, можно делить по возрастным группам |
| education |object | категориальное значение (образование), связано с 'education_id' |
| education_id |int64 | категориальное значение (образование), связано с 'education' |
| family_status |object | категориальное значение (семейное положение), связано с 'family_status_id' |
| family_status_id | nt64 | категориальное значение (семейное положение), связано с 'family_status' |
| gender |object | категориальное значение (пол), должно быть 'M' или 'F' |
| income_type |object | категориальное значение (занятость), вероятно, будет правильно присвоить ID |
| debt |int64 | категориальное значение (имел ли задолженность), должно быть 0 или 1 |
| total_income |float64 | количественное значение (заработок), должно быть >= 0, имеются пропуски |
| purpose |object | категориальное значение (уель кредита), вероятно, будет правильно присвоить ID |


### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
# создаем копию данных, с которыми будем работать
scoring = data.copy(deep=True)
# данные в колонках ['days_employed','dob_years', 'education', 'education_id', 'gender', 'income_type', 'debt']
# не связаны с поставленной задачей, поэтому
# значимые пропуски остались только в колонке 'total_income'
print(scoring.isna().sum())

# заполняем пропуски в З/П медианой
scoring['total_income'] = scoring['total_income'].fillna(data['total_income'].median())

# в колонке 'children' имеются ошибочные значения -1 и 20
print('\nЗначения столбца \'children\':', scoring['children'].value_counts(), sep='\n')
# заменяем на наиболее вероятные
scoring['children'] = scoring['children'].replace(-1, 1)
scoring['children'] = scoring['children'].replace(20, 2)

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Значения столбца 'children':
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


### Вывод

Вероятнее всего клиенты банка не указывали уровень доходов, поэтому ~10% пропусков по З/П. Их правильнее заполнять медианой, а не средним значением, т.к. медиана лучше характеризует "среднестатистического" человека. Не значимые для поставленных вопросов данные в дальнейшем можно будет отбросить.
Так же считаю правильным заменить значения "-1" и "20" на "1" и "2" соответственно, т.к. скорее всего это опечатки. Сильных отклонений в статистике это не вызовет, т.к. таких значений ~0.5%

### Замена типа данных

In [3]:
# можно преобразовать данные в столце 'total_income' в int
scoring['total_income'] = scoring['total_income'].astype(int)
scoring.info()

### Вывод

Точность в З/П в несколько знаков после запятой излишняя. Вполне достаточно целого числа.

### Обработка дубликатов

In [4]:
# перед удалением лучше все строки привести к нижнему регистру
print('Количество дублей до смены регистра:', scoring.duplicated().sum())

# список названий столбцов со строковыми значениями
names_str_columns = list(scoring.select_dtypes(include=['object']).columns)

# приведение к нижнему регистру
for name in names_str_columns:
    scoring[name] = scoring[name].str.lower()
print('Количество дублей после смены регистра:', scoring.duplicated().sum())

# удаляем повторы
scoring = scoring.drop_duplicates().reset_index(drop=True)
print('Количество дублей после удаления:', scoring.duplicated().sum())

# теперь можно оставить только необходимые колонки
scoring = scoring.drop(['days_employed','dob_years', 'education', 'education_id', 'gender', 'income_type'], axis='columns')

Количество дублей до смены регистра: 54
Количество дублей после смены регистра: 71


### Вывод

Вероятнее всего повторы возникли из-за повторных отправок форм через сайт или из-за ошибки человека при заполнении. Избавляемся от таких методом drop_duplicates() и сбрасываем индексы, не сохраняя старых методом .reset_index(drop=True). На этом этапе считаю, что можно удалить лишние колонки.

### Лемматизация

In [5]:
from pymystem3 import Mystem
m = Mystem()
# Получение лемм и их запись в новый столбец
# обрабатываем построчно объект series (scoring['purpose']) с помощью лямбда-функции ('x' - отдельная строка из колонки 'purpose')
scoring['lemmas'] = scoring['purpose'].apply(lambda x: m.lemmatize(x))
scoring

,children,family_status,family_status_id,debt,total_income,purpose,lemmas
0,1,женат / замужем,0,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,женат / замужем,0,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,женат / замужем,0,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,женат / замужем,0,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,гражданский брак,1,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"
...,...,...,...,...,...,...,...
21449,1,гражданский брак,1,0,224791,операции с жильем,"[операция, , с, , жилье, \n]"
21450,0,женат / замужем,0,0,155999,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]"
21451,1,гражданский брак,1,1,89672,недвижимость,"[недвижимость, \n]"
21452,3,женат / замужем,0,1,244093,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]"


### Вывод

С помощью модуля 'Mystem' библиотеки 'pymystem3' можно определить леммы для каждой строки с целью кредита, что в дальнейшем поможет с точным определением категорий.

### Категоризация данных

In [6]:
# определение возможных категорий
from collections import Counter

# оцениваем список и выделяем ключевые
print('Список всех лемм:\n', Counter(scoring['lemmas'].sum()))

# ф-ия присвоения категорий
def category(list):
    # использую словарь, т.к. леммы и категории не совпадают
    # порядок категорий имеет значение, например для : "строительство жилья", "ремонт жильЮ", "строительство жилой недвижимости" и т.д.
    # считаю, что пока достаточно такой реализации
    dict = {
        'строительство': 'ипотека на строительство',
        'ремонт': 'потребительский кредит на ремонт',
        'жилой': 'ипотека',
        'жилье': 'ипотека',
        'автомобиль': 'автокредит',
        'образование': 'образовательный кредит',
        'свадьба': 'потребительский кредит на свадьбу',
        'недвижимость': 'кредит на недвижимость',        
    }
    for word in list:
        if word in dict:
            return dict[word] # при первом найденном выходим из ф-ии
    return 'not_define' # для проверки охвата всех вариантов лемм

# добавляем столбец с именами категорий
scoring['purpose_category'] = scoring['lemmas'].apply(lambda x: category(x))
# проверка, остались ли некатегорированные данные
print('\nСписок итоговых категорий:\n', Counter(scoring['purpose_category']))

# разделим также клиентов по уровню З/П на категории от 1 до 10 по деци́лям.
# категория 1 будет соответствовать первому децилю (10% людей с наименьшим доходом)
# категория 10 будет соответствовать 10-му децилю (10% людей с наибольшим доходом)
# используем для создания новых колонок ф-цию "pd.qcut"
scoring['total_income_deciles'] = pd.qcut(scoring['total_income'], 10)
scoring['total_income_deciles_id'] = pd.qcut(scoring['total_income'], 10, labels=range(1,11))


Список всех лемм:
 Counter({' ': 33570, '\n': 21454, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})

Список итоговых категорий:
 Counter({'ипотека': 4459, 'автокредит': 4306, 'образовательный кредит': 4013, 'кредит на недвижимость': 3867, 'потребительский кредит на свадьбу': 2324, 'ипотека на строительство': 1878, 'потребительский кредит на ремонт': 607})


### Вывод

Исходя из статистики встречаемости лемм считаю правильным выделить ключевые:
- 'недвижимость': 6351
- 'жилье': 4460
- 'автомобиль': 4306
- 'образование': 4013
- 'свадьба': 2324
- 'строительство': 1878
- 'жилой': 1230
- 'ремонт': 607

Им в соответствием можно поставить 7 основных категорий. При этом сочетание лемм 'жилой' и 'недвижимость' можно определить как категорию 'ипотека'

Дополнительно создали категории по децилям колонки "total_income"

### Шаг 3. Ответьте на вопросы

#### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
# формируем сводную таблицу по кол-ву детей и фактом задолденности по кредиту
children_pt = scoring.pivot_table(scoring, index=['children','debt'], aggfunc={'debt':['count']})
# добавляем к ней колонку с подсчетом процента среди клиентов с таким же количеством детей
children_pt['percent'] = 100 * children_pt[('debt', 'count')] / scoring.groupby('children')['debt'].count()
# для наглядности выводим только "честных" клиентов в порядке убывания долей
children_pt.query('debt == 0').sort_values(axis=0, by='percent', ascending=False)

,,debt,percent
,,count,
children,debt,,
5,0,9,100.000000
0,0,13028,92.456178
3,0,303,91.818182
1,0,4410,90.834192
2,0,1926,90.507519
4,0,37,90.243902


### Вывод

Самыми платежеспособными оказались клиенты с 5-ю детьми, но таких всего лишь 9 человек, поэтому это не объективная статистика.

Люди без детей  немного чаще возвращаю кредиты в установленные сроки. "Честных" среди них ~92.46%.

Клиенты с 3-мя детьми возвращают кредит в срок чаще, чем клиенты с 1,2 и 4-мя детьми. (Таких записей 303 шт., возможно маткапитал на 3-х решает :-D, а может быть дело в сознательности таких людей).

Хуже всего дела обстоят у людей с 4-мя детьми. (Зато в старости дети будут обеспечивать... XD)

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [14]:
# формируем сводную таблицу по семейному положению и фактом задолденности по кредиту
family_status_pt = scoring.pivot_table(scoring, index=['family_status','debt'], aggfunc={'debt':['count']})
# добавляем к ней колонку с подсчетом процента среди клиентов с таким же семейным положением
family_status_pt['percent'] = 100 * family_status_pt[('debt', 'count')] / scoring.groupby(['family_status'])['debt'].count()
# для наглядности выводим только "честных" клиентов в порядке убывания долей
family_status_pt.query('debt == 0').sort_values(axis=0, by='percent', ascending=False)

,,debt,percent
,,count,
family_status,debt,,
вдовец / вдова,0,896,93.430657
в разводе,0,1110,92.887029
женат / замужем,0,11408,92.454818
гражданский брак,0,3763,90.652855
не женат / не замужем,0,2536,90.249110


### Вывод

Клиенты с семейным статусом "вдовец/вдова", "в разводе" или "женат/замужем" примерно на 2% чаще выплачивают кредит в срок, чем клиенты никогда не состоявшие в официальном браке

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [11]:
# формируем сводную таблицу по децилям З/П и фактом задолденности по кредиту
total_income_deciles_pt = scoring.pivot_table(scoring, index=['total_income_deciles', 'total_income_deciles_id','debt'], aggfunc={'debt':['count']})
# добавляем к ней колонку с подсчетом процента среди клиентов с таким же З/П
total_income_deciles_pt['percent'] = 100 * total_income_deciles_pt[('debt', 'count')] / scoring.groupby('total_income_deciles')['debt'].count()
# для наглядности выводим только "честных" клиентов в порядке убывания долей
print(total_income_deciles_pt.query('debt == 0').sort_values(axis=0, by='percent', ascending=False))

# количество клиентов в каждом дециле должно быть примерно одинаковое, но имеется сильное отклонение между 5 и 6 отрезком
# предположу, что это из-за заполнения пропусков в исходных данных по З/П медианой, которая как раз равняется 145017
print('\nКоличество значений "145017" в колонке "total_income":', scoring[scoring['total_income'] == scoring['total_income'].median()]['total_income'].count())

                                                    debt    percent
                                                   count           
total_income_deciles  total_income_deciles_id debt                 
(214606.0, 269797.4]  9                       0     1996  93.053613
(269797.4, 2265604.0] 10                      0     1995  92.963653
(20666.999, 78721.1]  1                       0     1989  92.684063
(116008.6, 135448.4]  4                       0     1970  91.798695
(98537.6, 116008.6]   3                       0     1966  91.655012
(179794.8, 214606.0]  8                       0     1964  91.561772
(135448.4, 145017.0]  5                       0     2922  91.398186
(78721.1, 98537.6]    2                       0     1958  91.282051
(156425.8, 179794.8]  7                       0     1957  91.192917
(145017.0, 156425.8]  6                       0      996  91.125343

Количество значений "145017" в колонке "total_income": 2104


### Вывод

Клиенты с самой высокой (9 и 10 дециль) и самой низкой зарплатой (1 дециль) чаще остальных выплачивают кредит в срок (болей 92%). Остальные значения примерно на 1% меньше.

#### Как разные цели кредита влияют на его возврат в срок?

In [10]:
# формируем сводную таблицу по категориям кредита и фактом задолденности по кредиту
purpose_category_pt = scoring.pivot_table(scoring, index=['purpose_category','debt'], aggfunc={'debt':['count']})
# добавляем к ней колонку с подсчетом процента среди клиентов с такой же категорией кредита
purpose_category_pt['percent'] = 100 * purpose_category_pt[('debt', 'count')] / scoring.groupby('purpose_category')['debt'].count()
# для наглядности выводим только "честных" клиентов в порядке убывания долей
purpose_category_pt.query('debt == 0').sort_values(axis=0, by='percent', ascending=False)

,,debt,percent
,,count,
purpose_category,debt,,
потребительский кредит на ремонт,0,572,94.233937
ипотека,0,4145,92.958062
кредит на недвижимость,0,3578,92.526506
ипотека на строительство,0,1734,92.332268
потребительский кредит на свадьбу,0,2138,91.996558
образовательный кредит,0,3643,90.779965
автокредит,0,3903,90.640966


### Вывод

Потребительские кредиты на ремонт гораздо чаще возвращают в срок. А образовательные кредиты и автокредиты - реже.

### Шаг 4. Общий вывод

Люди без детей  немного чаще возвращаю кредиты в установленные сроки. "Честных" среди них ~92.46%. Чем больше детей, тем меньше процент возврата.

Клиенты с семейным статусом "вдовец/вдова", "в разводе" или "женат/замужем" примерно на 2% чаще выплачивают кредит в срок, чем клиенты никогда не состоявшие в официальном браке.

Клиенты с самой высокой (9 и 10 дециль) и самой низкой зарплатой (1 дециль) чаще остальных выплачивают кредит в срок (болей 92%). Остальные значения примерно на 1% меньше.

Потребительские кредиты на ремонт гораздо чаще возвращают в срок. А образовательные кредиты и автокредиты - реже.